In [ ]:
from data_definition import load_and_transform, create_demand
import pyomo.environ as pyo

# Data import

In [ ]:
nodes, channels = load_and_transform("../data/network_graph_2024_06_12.json")
nodes = create_demand(nodes)

## Modeling

In [ ]:
model = pyo.ConcreteModel(name="Min cost flow problem")
model.NODES = pyo.Set(initialize=nodes.index)
model.CHANNELS = pyo.Set(initialize=channels.index) #within=model.NODES*model.NODES)

In [ ]:
model.x = pyo.Var(model.CHANNELS, domain=pyo.Binary)
model.a = pyo.Var(model.CHANNELS, domain=pyo.NonNegativeReals)

In [ ]:
def objective_function(model: pyo.ConcreteModel):
    return sum(channels.loc[k, "base_fee"] * model.x[k] for k in model.CHANNELS) + sum(channels.loc[i, "rate_fee"] * model.a[i] for i in model.CHANNELS)

model.totalCost = pyo.Objective(expr=objective_function(model), sense=pyo.minimize)

### Constraints

#### Capacity constraint

$$amount_{i,j} \le capacity_{i,j} \times x_{i,j} \text{ } \forall (i,j) \in E$$


In [ ]:
def capacity_constraint(model: pyo.ConcreteModel, a):
    return model.a[a] <= channels.loc[a, "capacity"] * model.x[a]

model.CapacityConstraint = pyo.Constraint(model.CHANNELS, rule=capacity_constraint, name="Capacity constraint")

#### Flow balance constraint

$$\sum_{(s,i) \in E} amount_{si} - \sum_{(i,t) \in E} amount_{it} = b_i \text{ } \forall i \in V$$


In [ ]:
def compute_outgoing(n: str) -> list:
    """
    Compute outgoing channels list for the node n
    :param n: node identifier
    :return: list of outgoing channels for node n
    """
    return [c for c in model.CHANNELS if channels.loc[c, "node1_pub"] == n]


def compute_incoming(n: str) -> list:
    """
    Compute incoming channels list for the node n
    :param n: node identifier
    :return: list of incoming channels for node n
    """
    return [c for c in model.CHANNELS if channels.loc[c, "node2_pub"] == n]


def flow_balance_constraint(model: pyo.ConcreteModel, n: str):
    print("Start processing")
    return sum(model.a[a] for a in compute_incoming(f"{n}")) - sum(model.a[a] for a in compute_outgoing(f"{n}")) == nodes.loc[n, "demand"]


model.FlowBalanceConstraint = pyo.Constraint(model.NODES, rule=flow_balance_constraint, name="Flow balance constrain")

## Solving the model

In [ ]:
opt = pyo.SolverFactory('cbc')
opt.solve(model, tee=True)

In [ ]:
model.x.pprint()